In [3]:
%pip install -q -U transformers
!git clone https://github.com/olgakanishcheva/Ukrainian-Text-Simplification.git -q
!git clone http://github.com/feralvam/easse.git -q
!cd easse && pip install . -q

Note: you may need to restart the kernel to use updated packages.
/bin/bash: pip: command not found


In [2]:
import torch
import html
import os
from easse.sari import corpus_sari
from easse.bleu import corpus_bleu
from easse.fkgl import corpus_fkgl

/home/artorius/miniconda3/envs/muss_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_dir='ASSETUKR/'
input_dir= 'simplified/'
dpl = os.path.join(input_dir,'muss-uk-dpl.txt')
gt = os.path.join(input_dir,'muss-uk-gt.txt')
sher = os.path.join(input_dir,'sherlock.txt')
sher_1st = os.path.join(input_dir,'sherlock-1st-sent.txt')
sher_tuned = os.path.join(input_dir,'sherlock-tuned.txt')
sher_tuned_1st = os.path.join(input_dir,'sherlock-tuned-1st-sent.txt')
spivavtor = os.path.join(input_dir,'spivavtor.txt')
spivavtor_tuned = os.path.join(input_dir,'spivavtor-tuned.txt')

In [4]:
def get_dataset(split, root_dir):
    orig_file = os.path.join(data_dir, f"asset.{split}.orig_uk.txt")
    simp_files = [os.path.join(data_dir, f"asset.{split}.simp.{i}_uk.txt") for i in range(10)]

    with open(orig_file, "r", encoding="utf-8") as orig_f:
        originals = [line.strip() for line in orig_f]

    simplifications = []
    for simp_file in simp_files:
        with open(simp_file, "r", encoding="utf-8") as simp_f:
            smp = [line.strip() for line in simp_f]
            # Select only simplifications which length match the original file length
            if len(smp) == len(originals):
                simplifications.append(smp)
                
    return (originals, simplifications)

In [5]:
originals, simplifications = get_dataset('test', data_dir)

In [6]:
def file_to_list(file_path):
    output=[]
    with open(file_path,'r') as file:
        return [line.strip() for line in file]

In [15]:
generates_results={
    # 'muss-dpl': file_to_list(dpl),
    # 'muss-gt': file_to_list(gt),
    # 'sherlock': file_to_list(sher),
    'sherlock-tuned': file_to_list(sher_tuned),
    'sherlock-tuned-1st-sent': file_to_list(sher_tuned_1st),
    'sherlock-1st-sent': file_to_list(sher_1st),
    'sherlock': file_to_list(sher),
    
    # 'spivavtor': file_to_list(spivavtor),
    # 'spivavtor-tuned': file_to_list(spivavtor_tuned)
}

In [13]:
for key, value in generates_results.items():
    sari_score = corpus_sari(orig_sents=originals,  
            sys_sents=value, 
            refs_sents=simplifications)
    bleu_score = corpus_bleu(
        sys_sents=value, 
        refs_sents=simplifications)
    print(f'{key} Sari score: {sari_score}') 
    print(f'{key} BLEU Score: {bleu_score}\n')
    # TODO: self-Bleu score
    # TODO: BertScore

dpl Sari score: 43.98811096581827
dpl BLEU Score: 49.88772290416101

gt Sari score: 43.047427749745246
gt BLEU Score: 43.07935504129747



In [8]:
len(generates_results['sherlock-tuned'])

359

In [16]:
import pandas as pd

df = pd.DataFrame(columns=['Model', 'SARI', 'BLEU'])
for key, value in generates_results.items():
    sari_score = corpus_sari(orig_sents=originals,  
            sys_sents=value, 
            refs_sents=simplifications)
    bleu_score = corpus_bleu(
        sys_sents=value, 
        refs_sents=simplifications)
    df = df.append({'Model': key, 'SARI': sari_score, 'BLEU': bleu_score}, ignore_index=True)

In [17]:
df

,Model,SARI,BLEU
0,sherlock-tuned,29.026455,13.920342
1,sherlock-tuned-1st-sent,31.511215,70.734145
2,sherlock-1st-sent,41.703331,37.078327
3,sherlock,41.678223,32.507096


In [18]:
from easse.bertscore import corpus_bertscore

# Assume `df` is your DataFrame and `generates_results` is a dictionary containing generated results

# Ensure your DataFrame has the necessary columns for BERTScore
if 'BERT_Precision' not in df.columns:
    df['BERT_Precision'] = None
if 'BERT_Recall' not in df.columns:
    df['BERT_Recall'] = None
if 'BERT_F1' not in df.columns:
    df['BERT_F1'] = None

# Loop through the generated results and calculate BERTScore
for key, value in generates_results.items():
    precision, recall, f1 = corpus_bertscore(sys_sents=value, refs_sents=simplifications)
    print(f'{key} BERTScore: (Precision: {precision}, Recall: {recall}, F1: {f1})\n')
    
    # Update the DataFrame with calculated values
    df.loc[df['Model'] == key, 'BERT_Precision'] = precision
    df.loc[df['Model'] == key, 'BERT_Recall'] = recall
    df.loc[df['Model'] == key, 'BERT_F1'] = f1

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


sherlock-tuned BERTScore: (Precision: 0.16594374179840088, Recall: 0.4190678596496582, F1: 0.27975913882255554)



Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


sherlock-tuned-1st-sent BERTScore: (Precision: 0.8363061547279358, Recall: 0.8178226351737976, F1: 0.8175695538520813)



Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


sherlock-1st-sent BERTScore: (Precision: 0.7422866225242615, Recall: 0.7179567813873291, F1: 0.7177876234054565)



Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


sherlock BERTScore: (Precision: 0.7279994487762451, Recall: 0.7185714244842529, F1: 0.710978090763092)



In [16]:
precision_dpl, recall_dpl, f1_dpl = precision, recall, f1

In [18]:
fkgl_dpl = corpus_fkgl(generates_results['dpl'])

In [19]:
df

,Model,SARI,BLEU,BERT_Precision,BERT_Recall,BERT_F1
0,sherlock-tuned,29.026455,13.920342,0.165944,0.419068,0.279759
1,sherlock-tuned-1st-sent,31.511215,70.734145,0.836306,0.817823,0.81757
2,sherlock-1st-sent,41.703331,37.078327,0.742287,0.717957,0.717788
3,sherlock,41.678223,32.507096,0.727999,0.718571,0.710978
